In [0]:
import warnings
warnings.filterwarnings("ignore")

In [0]:
from IPython.display import clear_output

In [0]:
!pip install pycodestyle flake8 pycodestyle_magic
%load_ext pycodestyle_magic
clear_output()

In [0]:
!pip install natasha
clear_output()

In [0]:
!pip install joblib
clear_output()

Задача: научиться находить в тексте лексически/стилистически неправильные сочетания (коллокации) и предлагать более правильные замены. <br>
Алгоритм является улучшенной версией этого алгоритма https://github.com/annadmitrieva/NLP-stuff/blob/master/collocation_generation-Copy4.ipynb 

#Этапы работы

*   Автоматическое обнаружение "неправильных" коллокаций в тексте.
*   Поиск замен.

Скачиваем word2vec модель из rusvectores.

In [0]:
!wget https://rusvectores.org/static/models/rusvectores4/ruwikiruscorpora/ruwikiruscorpora_upos_skipgram_300_2_2018.vec.gz
clear_output()

Для определения домена текста используем предобученную на шестиграммах из каждого домена (по 100000 на домен) SVM. F1-score модели – 0.94. <br>
Код модели: https://github.com/annadmitrieva/NLP-stuff/blob/master/domain_model.ipynb

In [0]:
!wget https://www.dropbox.com/s/lyfk0ur0pqvcgqv/domain_model.pkl?dl=0 -O domain_model.pkl
clear_output()

In [0]:
!wget https://www.dropbox.com/s/y35dm5chuv54psg/suggestions.zip?dl=0 -O suggestions.zip
clear_output()

In [0]:
!unzip suggestions.zip
clear_output()

In [0]:
import pandas as pd
import re
import nltk
import gensim
import joblib
import pickle
from gensim.models import Word2Vec
from natasha import NamesExtractor, AddressExtractor, DatesExtractor

model = gensim.models.KeyedVectors.load_word2vec_format(
    'ruwikiruscorpora_upos_skipgram_300_2_2018.vec.gz', binary=False)
domain_model = joblib.load('domain_model.pkl')

In [0]:
nltk.download('averaged_perceptron_tagger_ru')
nltk.download('punkt')
clear_output()

In [0]:
def domain(string):
    domain = domain_model.predict(list(string))[0]
    df = pd.read_csv(f'suggestions_{domain}.csv')
    return df

#Поиск неправильных сочетаний
В предыдущей версии этого алгоритма (https://github.com/annadmitrieva/NLP-stuff/blob/master/collocation_generation-Copy4.ipynb) в список плохих коллокаций попадали все встретившиеся в тексте биграммы из существительных или глаголов, которых не было в эталонном списке. В результате отлавливалось много пар слов, вообще не являющихся словосочетаниями (пример – "работы разрешение"). Решение: добавить синтаксический парсер. <br>
Будем считать плохими коллокациями такие пары "вершина-зависимое", которые состоят из существительных и глаголов и которых нет в эталонных списках правильных словосочетаний (для каждого домена текста – свой список).<br>
Эталонные списки коллокаций: https://drive.google.com/drive/folders/1k_N-DZ-nLL5ro66-LxIaE4-dRwirdwZh <br>
Код, который использовался при сборе эталонных списков: https://github.com/MariaFjodorowa/catandthekittens/tree/develop/collocations/collocation_frequencies

Несколько плохих текстов, на которых будем проверять, насколько хорошо находятся плохие коллокации. Источник – примеры плохой лексической сочетаемости из домашнего задания по академическому письму 1 курса ОП "Фундаментальная и прикладная лингвистика" НИУ ВШЭ 2017 года.

In [0]:
bad_text_pol = """В то же время, Путин умолчал от египетского населения данные о том, 
насколько милитаризируется сама Россия и насколько выросли ее военные расходы. 
Обама улетел из Вашингтона на вертолете, а Байден - на поезде."""
bad_text_hist = """Маккензи ограничивается рамками исследования 1928 – 1943, исследуя преимущественно идеологию Коминтерна сталинской эпохи, 
перечисляет причины на то, чтобы рассматривать 1928 год как переломный. 
Александр Шубин посвящает свою книгу большей частью на исследование общественно-политических движений в послесталинский СССР, 
1953 – 85 годов. В строках читаются напряжение и недоверие власти."""
bad_text_ling = """Следует избегать наводящие вопросы. Тем не менее существовало несколько точек зрения о создании алфавита. 
Предлагая ту или иную гипотезу, нельзя основываться и ссылаться только на результаты своего собственного исследования. 
Необходимо внимательное изучение опыта других ученых. 
Термин морфология обычно соотносится к части грамматики языка. Однако этот термин в истории науки употреблялся и в совсем ином значении."""

##Подход 1: MaltParser (медленный и плохо работающий)
Для POS-тэггинга использовался метод pos_tag из NLTK для русского языка.<br>
Для синтаксического парсинга использовался MaltParser из NLTK с предобученной моделью для русского языка. <br>
Источник модели: http://corpus.leeds.ac.uk/mocky/ <br>

In [0]:
!wget http://corpus.leeds.ac.uk/tools/russian.mco
!wget http://maltparser.org/dist/maltparser-1.8.1.zip
!unzip maltparser-1.8.1.zip
clear_output()

In [0]:
!export MALT_PARSER=$HOME/maltparser-1.8.1/
!export MALT_MODEL=$HOME/russian.mco

In [0]:
def pos_tag_rus(tokens):
    return nltk.pos_tag(tokens, lang='rus')

In [0]:
syntax_parser = nltk.parse.malt.MaltParser(
    'maltparser-1.8.1', 'russian.mco', tagger=pos_tag_rus)

In [0]:
# проверим, правильно ли работает парсер
sents = nltk.sent_tokenize(bad_text_pol)
nodes = syntax_parser.parse_one(sents[0].split()).nodes
for node in nodes.keys():
    head = nodes[node]['head']
    if head is not None and nodes[head]['word'] is not None:
        print(f"Head: {nodes[head]['word']} {nodes[head]['ctag']}. " +
              f"Dependent: {nodes[node]['word']} {nodes[node]['ctag']}.")

Head: то A-PRO=n. Dependent: В PR.
Head: то A-PRO=n. Dependent: же PART.
Head: то A-PRO=n. Dependent: время, S.
Head: время, S. Dependent: Путин S.
Head: от PR. Dependent: умолчал V.
Head: Путин S. Dependent: от PR.
Head: населения S. Dependent: египетского A=n.
Head: то A-PRO=n. Dependent: населения S.
Head: населения S. Dependent: данные S.
Head: том, S. Dependent: насколько ADV.
Head: сама A-PRO=f. Dependent: милитаризируется V.
Head: насколько ADV. Dependent: и CONJ.
Head: выросли V. Dependent: насколько ADV.
Head: выросли V. Dependent: ее S.
Head: ее S. Dependent: военные A=pl.
Head: военные A=pl. Dependent: расходы. S.


In [0]:
def search(text, collocations):
    """text – строка, collocations – датафрейм pandas.
    Возвращается словарь candidates,
    где ключи – пары слов, а значения – пары тэгов"""
    candidates = {}
    first = list(collocations.first_word)
    second = list(collocations.second_word)
    clean_text = re.sub("\([a-zA-z\d .,:]*\)|[a-zA-z\d]*", "", text)
    # чистим текст от библиографических ссылок
    # и токенов, состоящих только из латиницы и цифр
    extractors = [NamesExtractor(), AddressExtractor(), DatesExtractor()]
    named_entities = []
    # составляем список всех именованных сущностей, встречающихся в тексте
    for extractor in extractors:
        matches = extractor(text)
        for match in matches:
            start, stop = match.span
            for i in re.findall('\w+', text[start: stop]):
                named_entities.append(i.lower())
    # разбиваем текст на предложения, а предложения на слова
    sents = nltk.sent_tokenize(text)
    for i in range(len(sents)):
        nodes = syntax_parser.parse_one(sents[i].split()).nodes
        for node in nodes.keys():
            head = nodes[node]['head']
            if head is not None and nodes[head]['word'] is not None \
                    and nodes[head]['word'].lower() not in named_entities \
                    and nodes[node]['word'].lower() not in named_entities \
                    and nodes[head]['ctag'] in ['S', 'V'] \
                    and nodes[node]['ctag'] in ['S', 'V']:
                word_1 = nodes[head]['word'].lower()
                word_2 = nodes[node]['word'].lower()
                tag_1 = nodes[head]['ctag']
                tag_2 = nodes[node]['ctag']
                if word_1 in first or word_2 in second:
                    if word_1 in first and word_2 in second:
                        indices_1 = [i for i, x in enumerate(first)
                                     if x == word_1]
                        indices_2 = [i for i, x in enumerate(second)
                                     if x == word_2]
                        if not set(indices_1).isdisjoint(indices_2):
                            pass
                        else:
                            candidates[(word_1, word_2)] = (tag_1, tag_2)
                    else:
                        candidates[(word_1, word_2)] = (tag_1, tag_2)
    return candidates

Проверим, как функция search работает на наших плохих текстах.

In [0]:
%%time
candidates_pol = search(bad_text_pol, pd.read_csv('suggestions_pol.csv'))

CPU times: user 467 ms, sys: 62 ms, total: 529 ms
Wall time: 10.8 s


In [0]:
candidates_pol

{('населения', 'данные'): ('S', 'S')}

In [0]:
%%time
candidates_hist = search(bad_text_hist, pd.read_csv('suggestions_hist.csv'))

CPU times: user 252 ms, sys: 66 ms, total: 318 ms
Wall time: 16.3 s


In [0]:
candidates_hist

{('недоверие', 'власти.'): ('S', 'S'),
 ('ограничивается', 'рамками'): ('V', 'S'),
 ('перечисляет', 'причины'): ('V', 'S'),
 ('рамками', 'исследования'): ('S', 'S')}

In [0]:
%%time
candidates_ling = search(bad_text_ling, pd.read_csv('suggestions_ling.csv'))

CPU times: user 448 ms, sys: 142 ms, total: 590 ms
Wall time: 31.7 s


In [0]:
candidates_ling

{('зрения', 'создании'): ('S', 'S'),
 ('изучение', 'опыта'): ('S', 'S'),
 ('наводящие', 'избегать'): ('V', 'V'),
 ('наводящие', 'следует'): ('V', 'V'),
 ('создании', 'алфавита.'): ('S', 'S'),
 ('термин', 'морфология'): ('S', 'S')}

Результаты не очень хорошие – выделенные пары либо вообще не являются словосочетаниями ("населения данные", "зрения создании"), либо являются правильными словосочетаниями ("недоверие власти", "термин морфология"), при этом ни одно из спорных словосочетаний ("умолчал данные", "избегать вопросы") не выделены. Во многом эти ошибки вызваны плохой работой синтаксического парсера. <br>
Кроме того, MaltParser довольно медленный.

## Подход 2: StanfordNLP
Попробуем другой парсер зависимостей. <br>
Источник: https://github.com/stanfordnlp/stanfordnlp. <br>
Зависимостный парсер в StanfordNLP работает лучше и быстрее (см. код ниже) MaltParser, кроме того, в него уже встроен морфологический тэггер с тэгом PROPN, поэтому не нужно искать какой-то ещё тэггер или дополнительно убирать именованные сущности.

In [0]:
!pip install stanfordnlp
clear_output()

In [0]:
import stanfordnlp
stanfordnlp.download('ru')
clear_output()

In [0]:
nlp = stanfordnlp.Pipeline(lang='ru', models_dir='/root/stanfordnlp_resources')
clear_output()

In [0]:
doc = nlp(bad_text_pol)

In [18]:
# посмотрим, как работает этот парсер
doc.sentences[0].dependencies[0: 3]

[(<Word index=4;text=время;lemma=время;upos=NOUN;xpos=_;feats=Animacy=Inan|Case=Acc|Gender=Neut|Number=Sing;governor=7;dependency_relation=parataxis>,
  'case',
  <Word index=1;text=В;lemma=в;upos=ADP;xpos=_;feats=_;governor=4;dependency_relation=case>),
 (<Word index=4;text=время;lemma=время;upos=NOUN;xpos=_;feats=Animacy=Inan|Case=Acc|Gender=Neut|Number=Sing;governor=7;dependency_relation=parataxis>,
  'det',
  <Word index=2;text=то;lemma=тот;upos=DET;xpos=_;feats=Case=Acc|Gender=Neut|Number=Sing;governor=4;dependency_relation=det>),
 (<Word index=2;text=то;lemma=тот;upos=DET;xpos=_;feats=Case=Acc|Gender=Neut|Number=Sing;governor=4;dependency_relation=det>,
  'discourse',
  <Word index=3;text=же;lemma=же;upos=PART;xpos=_;feats=_;governor=2;dependency_relation=discourse>)]

In [0]:
# Так как в наших списках коллокаций используется немного другой набор тэгов,
# лучше сразу заменять NOUN на S, а VERB на V
tags = {'NOUN': 'S', 'VERB': 'V'}

def search(text, collocations):
    """text – строка, collocations – датафрейм pandas.
    Возвращается словарь candidates,
    где ключи – пары слов, а значения – пары тэгов"""
    candidates = {}
    first = list(collocations.first_word)
    second = list(collocations.second_word)
    clean_text = re.sub("\([a-zA-z\d .,:]*\)|[a-zA-z\d]*", "", text)
    # чистим текст от библиографических ссылок
    # и токенов, состоящих только из латиницы и цифр
    parsed_text = nlp(clean_text)
    for sent in parsed_text.sentences:
        for dep in sent.dependencies:
            if dep[0].upos in tags \
                    and dep[2].upos in tags:
                word_1 = dep[0].text.lower()
                word_2 = dep[2].text.lower()
                tag_1 = tags[dep[0].upos]
                tag_2 = tags[dep[2].upos]
                if word_1 in first or word_2 in second:
                    if word_1 in first and word_2 in second:
                        indices_1 = [i for i, x in enumerate(first)
                                     if x == word_1]
                        indices_2 = [i for i, x in enumerate(second)
                                     if x == word_2]
                        if not set(indices_1).isdisjoint(indices_2):
                            pass
                        else:
                            candidates[(word_1, word_2)] = (tag_1, tag_2)
                    else:
                        candidates[(word_1, word_2)] = (tag_1, tag_2)
    return candidates

In [21]:
%%time
candidates_pol = search(bad_text_pol, pd.read_csv('suggestions_pol.csv'))

CPU times: user 174 ms, sys: 17.1 ms, total: 191 ms
Wall time: 199 ms


In [22]:
candidates_pol

{('выросли', 'расходы'): ('V', 'S'),
 ('умолчал', 'время'): ('V', 'S'),
 ('умолчал', 'данные'): ('V', 'S'),
 ('умолчал', 'населения'): ('V', 'S')}

In [23]:
%%time
candidates_hist = search(bad_text_hist, pd.read_csv('suggestions_hist.csv'))

CPU times: user 223 ms, sys: 12 ms, total: 235 ms
Wall time: 242 ms


In [24]:
candidates_hist

{('исследование', 'движений'): ('S', 'S'),
 ('недоверие', 'власти'): ('S', 'S'),
 ('ограничивается', 'маккензи'): ('V', 'S'),
 ('ограничивается', 'перечисляет'): ('V', 'V'),
 ('ограничивается', 'рамками'): ('V', 'S'),
 ('перечисляет', 'причины'): ('V', 'S'),
 ('посвящает', 'книгу'): ('V', 'S'),
 ('посвящает', 'частью'): ('V', 'S'),
 ('рамками', 'исследования'): ('S', 'S'),
 ('рассматривать', 'год'): ('V', 'S'),
 ('частью', 'исследование'): ('S', 'S')}

In [25]:
%%time
candidates_ling = search(bad_text_ling, pd.read_csv('suggestions_ling.csv'))

CPU times: user 236 ms, sys: 7.97 ms, total: 244 ms
Wall time: 249 ms


In [26]:
candidates_ling

{('вопросы', 'наводящие'): ('S', 'V'),
 ('грамматики', 'языка'): ('S', 'S'),
 ('избегать', 'вопросы'): ('V', 'S'),
 ('изучение', 'опыта'): ('S', 'S'),
 ('опыта', 'ученых'): ('S', 'S'),
 ('предлагая', 'гипотезу'): ('V', 'S'),
 ('следует', 'избегать'): ('V', 'V'),
 ('создании', 'алфавита'): ('S', 'S'),
 ('соотносится', 'термин'): ('V', 'S'),
 ('соотносится', 'части'): ('V', 'S'),
 ('ссылаться', 'результаты'): ('V', 'S'),
 ('термин', 'истории'): ('S', 'S'),
 ('термин', 'морфология'): ('S', 'S'),
 ('точек', 'создании'): ('S', 'S'),
 ('употреблялся', 'значении'): ('V', 'S'),
 ('употреблялся', 'термин'): ('V', 'S'),
 ('части', 'грамматики'): ('S', 'S')}

Уже лучше! Все предложенные пары слов являются словосочетаниями (хотя функция *search()* всё ещё иногда ошибочно отлавливает правильные коллокации).

#Поиск и ранжирование возможных замен
Кандидаты на замену (не больше 10) подбираются по эталонному списку с учётом тэгов. Вес кандидата определяется по следующим метрикам: <br>

**Если кандидат на замену есть в эталонном списке:**
* **PMI (pointwise mutual information)**  – мера ассоциации между двумя словами, считается (по Manning and Schutze, 1999) как $\log\frac{P(x, y)}{P(x)P(y)}$, где $P(x)$ и $P(y)$ – вероятность появления в корпусе слов $x$ и $y$ соответственно, а $P(x, y)$ – вероятность появления их биграммы. Часто используется как метрика устойчивости словосочетаний. Может быть как положительной, так и отрицательной (но в наших эталонных списках PMI всегда положительный). Чем больше абсолютное значение PMI двух слов, тем больше они зависимы. PMI абсолютно независимых слов равна 0.<br>
* **Косинусная близость** (посчитанная с помощью word2vec) между правильным словом и словом на замену.

**Если в эталонном списке есть только оба слова кандидата по отдельности:**
* Близость слов в коллокационном кластере. Допустим, есть пара слов $N$ и $V$, у которых в эталонном списке есть следующие коллокаты: $N(V_{11}, V_{12}, V_{13})$, $V(N_{11}, N_{12}, N_{13})$. У данных коллокатов есть свои пары $V_{11}(N_{21}, N_{22})$, $V_{12}(N_{23}, N_{24})$ и т.д. Мера близости $N$ и $V$ по кластеру - это количество таких слов $N_{21}, N_{22} \ldots $, которые совпадают с коллокатами $V(N_{11}, N_{12}, N_{13})$, и таких слов $V_{21}, V_{22} \ldots $, которые совпадают с коллокатами $N(V_{11}, V_{12}, V_{13})$. Точно также она может считаться для пар $N$ и $N$ или $V$ и $V$.


Подробнее об этом алгоритме: https://aclweb.org/anthology/W09-2107 <br>
Проблема: функция *search()* всё ещё часто ошибочно находит правильные коллокации, которые не нужно заменять (например, "посвящает книгу", "термин морфология"). Решение: ввести дополнительный критерий отсева.  Например, считать PMI для "неправильного" словосочетания, и если он больше, чем у лучшего из кандидатов на замену, не заменять эту коллокацию.

При подсчёте PMI использовались предварительно подсчитанные вероятности вхождений всех униграмм и биграмм в корпус. <br>
Код, подсчитывающий вероятности: https://github.com/vyhuholl/cat_collocations/blob/master/uni_and_bigram_probability_counts.ipynb<br>
Результаты: <br>
https://www.dropbox.com/s/gxbmp952qwl03db/unigrams.pkl?dl=0 – для униграмм  <br>
 https://www.dropbox.com/s/o0f678z59mlu1d1/bigrams.pkl?dl=0 – для биграмм<br>
Частотные списки, использованные при подсчёте вероятностей: https://drive.google.com/drive/folders/1k_N-DZ-nLL5ro66-LxIaE4-dRwirdwZh

In [0]:
!wget https://www.dropbox.com/s/gxbmp952qwl03db/unigrams.pkl?dl=0 -O unigrams.pkl
!wget https://www.dropbox.com/s/o0f678z59mlu1d1/bigrams.pkl?dl=0 -O bigrams.pkl
clear_output()

In [0]:
with open("unigrams.pkl", "rb") as file:
    unigrams = pickle.load(file)
with open("bigrams.pkl", "rb") as file:
    bigrams = pickle.load(file)

In [0]:
from math import log


def PMI(word_1, word_2):
    if word_1 in unigrams and word_2 in unigrams \
            and f"{word_1} {word_2}" in bigrams:
        return log(bigrams[f"{word_1} {word_2}"] /
                   (unigrams[word_1] * unigrams[word_2]), 2)
    return 0

In [30]:
# посмотрим, правильно ли работает функция –
# для нескольких эталонных словосочетаний
# сравним PMI, посчитанное функцией
# и PMI, указанное в эталонных списках
print(PMI("реализацию", "носового")) # значение в эталонном списке – 10.948051
print(PMI("актором", "оформляет")) # значение в эталонном списке – 17.482237
print(PMI("адрес", "фон")) # значение в эталонном списке – 7.276692
print(PMI("хватает", "глубины")) # значение в эталонном списке – 13.626561
print(PMI("требованию", "истца")) # значение в эталонном списке – 10.709151
print(PMI("исследования", "ефимова")) # значение в эталонном списке – 7.723045
print(PMI("выбор", "работников")) # значение в эталонном списке – 4.089518

10.622390362568218
15.832522922888293
10.251731553169732
13.894368245176478
12.1509383953229
8.967164778531066
7.120048201210611


Значения PMI, посчитанные функцией, достаточно близки к тем, которые были посчитаны для эталонных списков.

Word2Vec модель из rusvectores работает только с леммами слов, к которым приклеены тэги. Поэтому нам понадобится pymorphy для лемматизации. 

In [0]:
!pip install pymorphy2
clear_output()

In [0]:
from pymorphy2 import MorphAnalyzer

morph = MorphAnalyzer()

In [0]:
def convert(word, tag):
    if tag=='S':
        return morph.parse(word)[0].normal_form+'_'+'NOUN'
    elif tag=='V':
        return morph.parse(word)[0].normal_form+'_'+'VERB'

In [0]:
def suggest(candidates, collocations):
    """candidates – словарь, где ключи – пары слов,
    а значения – пары тэгов, collocations –
    датафрейм pandas. Возвращается
    словарь corrections, где ключи –
    плохие коллокации, а значения – списки
    замен (пара "замена + её вероятность),
    упорядоченные от наиболее вероятной
    до наименее."""
    max_value = max(collocations.pmi) + 1
    corrections = dict()
    for bigram in candidates:
        max_pmi = 0
        suggestions = dict()
        word_1, word_2 = bigram[0], bigram[1]
        tag_1, tag_2 = candidates[bigram][0], candidates[bigram][1]
        # в words_1 и words_2 будут храниться те кандидаты
        # на замену первого и второго слова соответственно
        # которые встречаются в эталонном списке по отдельности,
        # не образуя коллокации
        words_1 = []
        words_2 = []
        for i in range(len(collocations) - 1):
            # попробуем заменить первое слово, если второе совпадает
            if collocations.first_tag[i] == tag_1 \
                    and collocations.second_word[i] == word_2:
                weight = collocations.pmi[i]
                if weight > max_pmi:
                    max_pmi = weight
                wv_1 = convert(collocations.first_word[i], tag_1)
                wv_2 = convert(word_1, tag_1)
                if wv_1 in model.wv and wv_2 in model.wv:
                    weight += model.wv.similarity(wv_1, wv_2)
                weight /= max_value
                suggestions[(collocations.first_word[i], word_2)] = weight
                # нормируем, чтобы все вероятности лежали в диапазоне от 0 до 1
                words_1.append(collocations.first_word[i])
            # теперь попробуем заменить второе слово, если первое совпадает
            if collocations.first_word[i] == word_1 \
                    and collocations.second_tag[i] == tag_2:
                weight = collocations.pmi[i]
                if weight > max_pmi:
                    max_pmi = weight
                wv_1 = convert(collocations.second_word[i], tag_2)
                wv_2 = convert(word_2, tag_2)
                if wv_1 in model.wv and wv_2 in model.wv:
                    weight += model.wv.similarity(wv_1, wv_2)
                weight /= max_value
                suggestions[(word_1, collocations.second_word[i])] = weight
                words_2.append(collocations.second_word[i])
        # если остались отдельные кандидаты
        # как на замену word_1, так и на замену word_2
        # то для них тоже считаем вероятности
        if len(words_1) > 0 and len(words_2) > 0:
            for word in words_1:
                for i in range(len(collocations) - 1):
                    first = collocations.first_word[i]
                    second = collocations.second_word[i]
                    if first == word and second in words_2:
                        if (word, second) not in suggestions:
                            suggestions[(word, second)] = 1.0 / len(words_2)
                        else:
                            suggestions[(word, second)] += 1.0 / len(words_2)
            for word in words_2:
                for i in range(len(collocations) - 1):
                    first = collocations.first_word[i]
                    second = collocations.second_word[i]
                    if first in words_1 and second == word:
                        if (first, word) not in suggestions:
                            suggestions[(first, word)] = 1.0 / len(words_1)
                        else:
                            suggestions[(first, word)] += 1.0 / len(words_1)
        best = [item for item in sorted(suggestions.items(),
                                        key=lambda x: x[-1],
                                        reverse=True)[:10]
                if item[1] >= 0.5]
        if max_pmi > PMI(word_1, word_2):
            corrections[bigram] = best
    return corrections

Проверим!

In [54]:
candidates_pol

{('выросли', 'расходы'): ('V', 'S'),
 ('умолчал', 'время'): ('V', 'S'),
 ('умолчал', 'данные'): ('V', 'S'),
 ('умолчал', 'населения'): ('V', 'S')}

In [55]:
%%time
suggestions_pol = suggest(candidates_pol, pd.read_csv('suggestions_pol.csv'))

CPU times: user 5.68 s, sys: 4.73 ms, total: 5.69 s
Wall time: 5.69 s


In [74]:
for bad in suggestions_pol:
    print(f"Bad: {bad}.")
    print("Suggestions:")
    print('\n'.join(f"{i[0]}, {i[1]}" for i in suggestions_pol[bad]))
    print("\n")

Bad: ('умолчал', 'время').
Suggestions:
('настало', 'время'), 0.5706590754770549
('потребуется', 'время'), 0.5621175967293761


Bad: ('умолчал', 'данные').
Suggestions:
('обобщая', 'данные'), 0.596687089644353


Bad: ('выросли', 'расходы').
Suggestions:
('оптимизирует', 'расходы'), 0.7035836770729458
('увеличили', 'расходы'), 0.6582008296985596
('требуются', 'расходы'), 0.6052636331236313
('свидетельствовать', 'расходы'), 0.5905498243138871




In [70]:
candidates_hist

{('исследование', 'движений'): ('S', 'S'),
 ('недоверие', 'власти'): ('S', 'S'),
 ('ограничивается', 'маккензи'): ('V', 'S'),
 ('ограничивается', 'перечисляет'): ('V', 'V'),
 ('ограничивается', 'рамками'): ('V', 'S'),
 ('перечисляет', 'причины'): ('V', 'S'),
 ('посвящает', 'книгу'): ('V', 'S'),
 ('посвящает', 'частью'): ('V', 'S'),
 ('рамками', 'исследования'): ('S', 'S'),
 ('рассматривать', 'год'): ('V', 'S'),
 ('частью', 'исследование'): ('S', 'S')}

In [71]:
%%time
suggestions_hist = suggest(candidates_hist, pd.read_csv('suggestions_hist.csv'))

CPU times: user 8.68 s, sys: 6.96 ms, total: 8.69 s
Wall time: 8.69 s


In [75]:
for bad in suggestions_hist:
    print(f"Bad: {bad}.")
    print("Suggestions:")
    print('\n'.join(f"{i[0]}, {i[1]}" for i in suggestions_hist[bad]))
    print("\n")

Bad: ('ограничивается', 'маккензи').
Suggestions:
('ограничивается', 'проведением'), 0.9506715008756121


Bad: ('ограничивается', 'рамками').
Suggestions:
('ограничивается', 'проведением'), 0.9639394046257748


Bad: ('рамками', 'исследования').
Suggestions:
('ракурса', 'исследования'), 0.7055136153085221
('объекта', 'исследования'), 0.705067710521165
('объектами', 'исследования'), 0.705067710521165
('достоинством', 'исследования'), 0.7005627433862813
('структура', 'исследования'), 0.621163092281545
('фонда', 'исследования'), 0.6176795702425913
('метода', 'исследования'), 0.6157198208569395
('теме', 'исследования'), 0.5319833779033427


Bad: ('перечисляет', 'причины').
Suggestions:
('объяснить', 'причины'), 0.6861508304288867
('рассматривает', 'причины'), 0.6591480444472243
('рассматриваются', 'причины'), 0.6309382150289891


Bad: ('рассматривать', 'год').
Suggestions:
('рассматривать', 'круги'), 0.6918590182218118
('рассматривать', 'власть'), 0.6191679544646704


Bad: ('посвящает', 'кн

In [76]:
candidates_ling

{('вопросы', 'наводящие'): ('S', 'V'),
 ('грамматики', 'языка'): ('S', 'S'),
 ('избегать', 'вопросы'): ('V', 'S'),
 ('изучение', 'опыта'): ('S', 'S'),
 ('опыта', 'ученых'): ('S', 'S'),
 ('предлагая', 'гипотезу'): ('V', 'S'),
 ('следует', 'избегать'): ('V', 'V'),
 ('создании', 'алфавита'): ('S', 'S'),
 ('соотносится', 'термин'): ('V', 'S'),
 ('соотносится', 'части'): ('V', 'S'),
 ('ссылаться', 'результаты'): ('V', 'S'),
 ('термин', 'истории'): ('S', 'S'),
 ('термин', 'морфология'): ('S', 'S'),
 ('точек', 'создании'): ('S', 'S'),
 ('употреблялся', 'значении'): ('V', 'S'),
 ('употреблялся', 'термин'): ('V', 'S'),
 ('части', 'грамматики'): ('S', 'S')}

In [77]:
%%time
suggestions_ling = suggest(candidates_ling, pd.read_csv('suggestions_ling.csv'))

CPU times: user 1min 22s, sys: 20.8 ms, total: 1min 22s
Wall time: 1min 22s


In [78]:
for bad in suggestions_ling:
    print(f"Bad: {bad}.")
    print("Suggestions:")
    print('\n'.join(f"{i[0]}, {i[1]}" for i in suggestions_ling[bad]))
    print("\n")

Bad: ('следует', 'избегать').
Suggestions:
('стремился', 'избегать'), 0.8212504884404777
('следует', 'уделять'), 0.6218461564194934
('следует', 'простить'), 0.6193272360438431
('следует', 'вознести'), 0.6170769625499726
('следует', 'применять'), 0.5716005921887101
('следует', 'определять'), 0.5702201276884263
('следует', 'увязать'), 0.5697844014774479
('следует', 'напомнить'), 0.5655767932970802
('следует', 'упомянуть'), 0.5654509246206123
('следует', 'отметить'), 0.5605653249470472


Bad: ('избегать', 'вопросы').
Suggestions:
('избегать', 'пересказа'), 0.8148288932660556
('избегать', 'запретов'), 0.770124670554275
('обсуждались', 'вопросы'), 0.5251686057974542


Bad: ('точек', 'создании').
Suggestions:
('точек', 'зрения'), 0.6324493110413522


Bad: ('создании', 'алфавита').
Suggestions:
('создании', 'ритма'), 0.6001357603246426
('создании', 'мия'), 0.5238329906833026
('создании', 'памятника'), 0.5218224846940478


Bad: ('предлагая', 'гипотезу').
Suggestions:
('проверим', 'гипотезу'), 